In [11]:
import numpy as np
import itertools
import time
from collections import Counter

In [12]:
scene = {
    (0,0): 'red,metal,sphere',
    (0,1):'green,matte,cube',
    (1,1):'blue,metal,cube'
}

In [13]:
def make_base_paths(scene):
    '''make base point A to B position paths from scene'''
    paths = set()
    for pos1, pos2 in itertools.product(*[scene.keys(), scene.keys()]):
        i1, j1 = pos1
        i2, j2 = pos2
        horizontal = i1 == i2
        vertical = j1 == j2
        
        path = None
        if horizontal and abs(j1-j2)==1:
            if j1 > j2:
                path = (pos1, 'RL', pos2)
            else:
                path = (pos1, 'LR', pos2)
        if vertical and abs(i1-i2)==1:
            if i1 > i2:
                path = (pos1, 'BA', pos2)
            else:
                path = (pos1, 'AB', pos2)
        if path: paths.add(path)
    return paths

In [14]:
def connect_paths(paths):
    '''connect base paths into longer paths'''
    new_paths = set()
    for path1, path2 in itertools.combinations(paths, 2):
        s1, e1 = path1[0], path1[-1]
        s2, e2 = path2[0], path2[-1]
        new_path = None
        
        # TODO check that path doesn't overlap itself?
        if (e1 == s2 and not(set(path1[::2][:-1]) & set(path2[::2][1:]))):
            new_path = (s1,) + path1[1:-1] + (e1,) + path2[1:-1] + (e2,)
        elif (s1 == e2 and not(set(path1[::2][1:]) & set(path2[::2][:-1]))):
            new_path = (s2,) + path2[1:-1] + (e2,) + path1[1:-1] + (e1,)
        if new_path: new_paths.add(new_path)
    return new_paths

connect_paths(make_base_paths(scene))

{((0, 0), 'LR', (0, 1), 'AB', (1, 1)), ((1, 1), 'BA', (0, 1), 'RL', (0, 0))}

In [15]:
def translate_paths(paths, scene):
    '''translate position paths into obj attr paths'''
    translated_paths = set()
    for p in paths:
#         translated_path = tuple('\('+scene[x]+'\)' if isinstance(x, tuple) else x for x in p)
#         translated_path = '-'.join(translated_path)
        translated_path = tuple(scene[x] if isinstance(x, tuple) else x for x in p)
        translated_paths.add(translated_path)
    return translated_paths
    
translate_paths(connect_paths(make_base_paths(scene)), scene)

{('blue,metal,cube', 'BA', 'green,matte,cube', 'RL', 'red,metal,sphere'),
 ('red,metal,sphere', 'LR', 'green,matte,cube', 'AB', 'blue,metal,cube')}

In [16]:
def swap_object_attr(p, attributes, num_swaps, debug=False):
    '''swap out num_swaps object attributes to make a negative example'''
    
    p = list(p)
    if debug: print(p)
        
    swap_pos_all = np.random.choice(list(range(len(p))[::2]), size=min(int(num_swaps), len(p)//2), replace=False)
    attr_pos_all = np.random.choice(3, size=int(num_swaps))

    for swap_pos, attr_pos in zip(swap_pos_all, attr_pos_all):
        if debug:
            print(swap_pos, p[swap_pos], attr_pos, p[swap_pos].split(',')[attr_pos])
        
        if attr_pos == 0:
            attr_options = attributes['colors']
        elif attr_pos == 1:
            attr_options = attributes['textures']
        else:
            attr_options = attributes['shapes']
        
        obj_attrs = p[swap_pos].split(',')
        old_attr = obj_attrs[attr_pos]
        new_attr = old_attr
        while new_attr == old_attr:
            print('while loop within swap_object_attr')
            new_attr = np.random.choice(attr_options)
        obj_attrs[attr_pos] = new_attr
        p[swap_pos] = ','.join(obj_attrs)

    if debug: print(p)
    return tuple(p)

In [17]:
def swap_relations(p, attributes, num_swaps, debug=False):
    '''swap out num_swaps object relations to make a negative example'''
    
    p = list(p)
    if debug: print(p)
    swap_pos_all = np.random.choice(list(range(len(p))[1::2]), size=min(int(num_swaps), len(p)//2), replace=False)
    attr_options = attributes['relations']
    for swap_pos in swap_pos_all:
        if debug:
            print(swap_pos, p[swap_pos])
        old_attr = p[swap_pos]
        new_attr = old_attr
        while new_attr == old_attr:
            print('while loop within swap_relations')
            new_attr = np.random.choice(attr_options)
        p[swap_pos] = new_attr
        
    if debug: print(p)
    return tuple(p)

In [25]:
for i in range(100):
    print(f'i = {i}')
    num = 0
    while num%2 == 0:
        print(f'num = {num}')
        num = np.random.choice(10)
        if num == 2:
            print('num == 2, breaking  here')
            break

i = 0
num = 0
i = 1
num = 0
i = 2
num = 0
i = 3
num = 0
num == 2, breaking  here
i = 4
num = 0
i = 5
num = 0
i = 6
num = 0
num = 4
i = 7
num = 0
i = 8
num = 0
num = 8
num = 0
i = 9
num = 0
i = 10
num = 0
num = 0
i = 11
num = 0
num = 4
num = 6
i = 12
num = 0
num == 2, breaking  here
i = 13
num = 0
num = 4
num == 2, breaking  here
i = 14
num = 0
num = 8
i = 15
num = 0
i = 16
num = 0
num = 8
i = 17
num = 0
num = 6
num = 4
i = 18
num = 0
i = 19
num = 0
i = 20
num = 0
i = 21
num = 0
i = 22
num = 0
i = 23
num = 0
i = 24
num = 0
num = 6
num = 6
num == 2, breaking  here
i = 25
num = 0
num = 6
i = 26
num = 0
num = 4
num = 0
num = 0
i = 27
num = 0
i = 28
num = 0
num = 6
num == 2, breaking  here
i = 29
num = 0
i = 30
num = 0
num = 8
i = 31
num = 0
i = 32
num = 0
num = 0
num == 2, breaking  here
i = 33
num = 0
i = 34
num = 0
i = 35
num = 0
num = 0
i = 36
num = 0
i = 37
num = 0
num = 4
i = 38
num = 0
i = 39
num = 0
num == 2, breaking  here
i = 40
num = 0
num = 8
num = 4
i = 41
num = 0
i = 42
num = 

In [33]:
def make_all_paths(scene, attributes, num_levels=3, num_swaps=1):
    
    # 3 levels of paths
    base_paths = make_base_paths(scene) # single straight edge
    gt_position_paths = base_paths
    for i in range(num_levels):
        gt_position_paths = gt_position_paths | connect_paths(gt_position_paths)
    
    # positive paths  
    positive_objattr_paths = translate_paths(gt_position_paths, scene)
    
    # sample negative paths by swapping
    negative_objattr_attr_paths = set()
    for pos_p_i, pos_p in enumerate(positive_objattr_paths):
        # either swap a relation or an attribute
        choice = np.random.choice(2)
        swap_fn = swap_relations if choice == 0 else swap_object_attr
        neg_p = pos_p
        retry = 0
        while neg_p in positive_objattr_paths:
            print(f'{pos_p_i} while loop within make_all_paths, rety={retry}')
            retry += 1
            if retry == 5:
                break 
            neg_p = swap_fn(pos_p, attributes, num_swaps)
            
        if neg_p not in positive_objattr_paths:
            negative_objattr_attr_paths.add(neg_p)
    
    assert not (positive_objattr_paths & negative_objattr_attr_paths)
    return gt_position_paths, positive_objattr_paths, negative_objattr_attr_paths

In [19]:
def sample_scene(num_slots, grid_dim, attributes):
    assert num_slots <= grid_dim**2
    all_positions = list((i,j) for i,j in itertools.product(range(grid_dim), range(grid_dim)))
    indices = np.random.choice(len(all_positions), num_slots, replace=False)
    scene_positions = list(all_positions[idx] for idx in indices)
    scene_shapes = np.random.choice(attributes['shapes'], size=num_slots)
    scene_colors = np.random.choice(attributes['colors'], size=num_slots)
    scene_materials = np.random.choice(attributes['textures'], size=num_slots)
    scene = {
        pos: ','.join([c,m,s]) for pos, c, m, s in \
        zip(scene_positions, scene_colors, scene_materials, scene_shapes)
    }
    metadata = {'grid_dim':grid_dim, 'num_slots':num_slots}
    return scene, metadata

In [20]:
def make_scene_input(scene, metadata):
    scene_input = []
    for i in range(metadata['grid_dim']):
        for j in range(metadata['grid_dim']):
            if (i,j) in scene:
                c, t, s = scene[(i,j)].split(',')
                scene_input.append((i, j, c, t, s))
            else:
                scene_input.append((i, j, 'none', 'none', 'none'))
    return scene_input

                
# ToDo input query format
def make_query_input(objattr_path):
    return '-'.join(f'({s})' if i%2==0 else s for i,s in enumerate(objattr_path))

In [21]:
def abbreviate(obj_attr_str):
    '''red,metal,cube to rmc'''
    return ''.join([s[0] for s in obj_attr_str.split(',')])

def plot_scene(scene, metadata):
    for i in range(metadata['grid_dim']):
        print(' - '.join([abbreviate(scene.get((i,j), ' , , ')) for j in range(metadata['grid_dim'])]))
        print('   '.join([f'{i},{j}' for j in range(metadata['grid_dim'])]))
        if not i == metadata['grid_dim']-1:
            print('   '.join([f' | ' for j in range(metadata['grid_dim'])]))

## Sample One Scene

In [17]:
attributes = {
    'colors': ['red', 'green', 'blue'],
    'textures': ['metal', 'rubber'],
    'shapes': ['cube', 'sphere', 'tetrahedron'],
    'relations': ['RL', 'LR', 'AB', 'BA']
}

np.random.seed(39)

scene, metadata = sample_scene(num_slots=18, grid_dim=6, attributes=attributes)

plot_scene(scene, metadata)

gt_position_paths, positive_objattr_paths, negative_objattr_attr_paths = make_all_paths(scene, attributes, num_levels=4, num_swaps=1)

    - grc - bms -     -     - grs
0,0   0,1   0,2   0,3   0,4   0,5
 |     |     |     |     |     | 
    - bmc - gmc -     - gms -    
1,0   1,1   1,2   1,3   1,4   1,5
 |     |     |     |     |     | 
    -     -     - brc -     -    
2,0   2,1   2,2   2,3   2,4   2,5
 |     |     |     |     |     | 
rmc - gms -     - gmt - gmc -    
3,0   3,1   3,2   3,3   3,4   3,5
 |     |     |     |     |     | 
gms -     - rrt -     - brc - grt
4,0   4,1   4,2   4,3   4,4   4,5
 |     |     |     |     |     | 
    - bms - rmc -     -     - rmc
5,0   5,1   5,2   5,3   5,4   5,5


In [18]:
print(make_scene_input(scene, metadata))
print('----------------------------------------')
print(make_query_input(('blue,metal,sphere', 'AB', 'green,metal,cube')))

[(0, 0, 'none', 'none', 'none'), (0, 1, 'green', 'rubber', 'cube'), (0, 2, 'blue', 'metal', 'sphere'), (0, 3, 'none', 'none', 'none'), (0, 4, 'none', 'none', 'none'), (0, 5, 'green', 'rubber', 'sphere'), (1, 0, 'none', 'none', 'none'), (1, 1, 'blue', 'metal', 'cube'), (1, 2, 'green', 'metal', 'cube'), (1, 3, 'none', 'none', 'none'), (1, 4, 'green', 'metal', 'sphere'), (1, 5, 'none', 'none', 'none'), (2, 0, 'none', 'none', 'none'), (2, 1, 'none', 'none', 'none'), (2, 2, 'none', 'none', 'none'), (2, 3, 'blue', 'rubber', 'cube'), (2, 4, 'none', 'none', 'none'), (2, 5, 'none', 'none', 'none'), (3, 0, 'red', 'metal', 'cube'), (3, 1, 'green', 'metal', 'sphere'), (3, 2, 'none', 'none', 'none'), (3, 3, 'green', 'metal', 'tetrahedron'), (3, 4, 'green', 'metal', 'cube'), (3, 5, 'none', 'none', 'none'), (4, 0, 'green', 'metal', 'sphere'), (4, 1, 'none', 'none', 'none'), (4, 2, 'red', 'rubber', 'tetrahedron'), (4, 3, 'none', 'none', 'none'), (4, 4, 'blue', 'rubber', 'cube'), (4, 5, 'green', 'rubbe

## Sample Many Scenes

In [35]:
np.random.seed(42)

num_scenes = 5000 # number of scenes
grid_dim = 3 # grid dimension
num_slots = 3 # number of slots to fill
num_levels = 3 # max path length = 2*4=16
num_swaps = 1  # number of attributes or relations to switch out for negative examples

attributes = {
    'colors': ['red', 'green', 'blue'],
    'textures': ['metal', 'rubber'],
    'shapes': ['cube', 'sphere', 'tetrahedron'],
    'relations': ['RL', 'LR', 'AB', 'BA']
}

start = time.time()
dataset = {}
for i in range(num_scenes):
    scene, metadata = sample_scene(num_slots, grid_dim, attributes)
    gt_position_paths, positive_objattr_paths, negative_objattr_attr_paths = make_all_paths(scene, attributes, num_levels, num_swaps)
    p_lens = []
    for p in gt_position_paths:
        p_lens.append(len(p)//2)
    
    dataset[i] = {
        'scene': scene,
        'metadata': metadata,
        'gt_position_paths': gt_position_paths,
        'positive_objattr_paths': positive_objattr_paths,
        'negative_objattr_attr_paths': negative_objattr_attr_paths,
        'num_paths': len(gt_position_paths),
        'path_len_ctr': Counter(p_lens)
    }
print(time.time() - start, ' seconds')

0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
2 while loop within make_all_paths, rety=0
while loop within swap_object_attr
3 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
4 while loop within make_all_paths, rety=0
while loop within swap_relations
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 w

0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
2 while loop within make_all_paths, rety=0
while loop within swap_relations
3 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
4 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_p

while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
2 while loop within ma

0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
2 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
3 while loop within make_all_paths, rety=0
while loop within swap_relations
4 while loop within make_all_paths, rety=0
while loop within swap_relations
5 while loop within make_all_paths, rety=0
while loop withi

1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
2 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
3 while loop within make_all_paths, rety=0
while loop within swap_object_attr
4 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within 

...

while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
2 while loop within make_all_paths, rety=0
while loop within swap_relations
3 while loop within make_all_paths, rety=0
while loop within swap_relations
4 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within 

while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop wit

while loop within swap_relations
2 while loop within make_all_paths, rety=1
while loop within swap_relations
2 while loop within make_all_paths, rety=2
while loop within swap_relations
2 while loop within make_all_paths, rety=3
while loop within swap_relations
2 while loop within make_all_paths, rety=4
3 while loop within make_all_paths, rety=0
while loop within swap_relations
3 while loop within make_all_paths, rety=1
while loop within swap_relations
3 while loop within make_all_paths, rety=2
while loop within swap_relations
3 while loop within make_all_paths, rety=3
while loop within swap_relations
3 while loop within make_all_paths, rety=4
4 while loop within make_all_paths, rety=0
while loop within swap_relations
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
whil

while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
2 while loop within make_all_paths, rety=0
while loop within swap_object_attr
3 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
while loop within swap_relations
4 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
while loop within swap_relat

while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
0 while loop within make_all_paths, rety=0
while loop within swap_object_attr
1 while loop within make_all_paths, rety=0
while loop within swap_relations
2 while loop within make_all_paths, rety=0
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
while loop within swap_object_attr
3 while loop within make_all_paths, rety=0
while loop within swap_relations
4 while loop within make_all_paths, rety=0
while loop within swap_relations
5 while loop within make_all_paths, rety=0
while loop within swap_object_attr
0 while loop within make_all_paths, rety=0
while loop within swap_relations
1 while loop within make_all_paths, rety=0
while loop within sw

In [ ]:
# count number of data points without paths

In [ ]:
# add path_len_ctr together

In [ ]:
# do queries from one set overlap with another?

In [ ]:
# E0D
# Get come statistics of scenes, path lenghts, labels
# get aggregate statistics to see how much queries overlap

In [ ]:
# ToDo While loop is stuck

In [ ]:
# Training
# Have inputs and outputs ready